# Pre-trained FastText Fine-Tunning
- ``cc.ko.300.bin``은 Facebook에서 wikipedia와 common crawler의 데이터로 사전 학습을 진행한 fasttext 모델이다.
    - 다운로드 링크 : https://fasttext.cc/docs/en/crawl-vectors.html

## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from gensim import models
from tqdm.notebook import tqdm

## 2. Pre-Trained FastText model 로드

In [2]:
ko_models = models.fasttext.load_facebook_model('./pre_trained_fasttext/cc.ko.300.bin')

## 3. Pre-Trained FastText model Fine-Tunning

### 1) Data set

#### (1) 판례 데이터 로드

In [3]:
prec_df = pd.read_csv('../data/prec_data.csv', encoding='utf-8-sig')
prec_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,20220415.0,선고,수원고법,400202.0,형사,400102.0,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,NaN,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,20220411.0,자,서울고등법원,400202.0,가사,400103.0,결정,NaN,NaN,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",NaN,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,20220323.0,선고,대구고법,400202.0,민사,400101.0,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",NaN,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,20220322.0,자,대법원,400201.0,가사,400103.0,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),NaN,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",NaN,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,20220317.0,선고,대법원,400201.0,민사,400101.0,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


#### (2) 판시사항 + 판결요지 + 판례내용 X 값 만들기

In [4]:
train_dataset = []

for idx in tqdm(range(len(prec_df))):
    x = ''
    if not pd.isna(prec_df['판시사항'][idx]):
        x = x + prec_df['판시사항'][idx]
    if not pd.isna(prec_df['판결요지'][idx]):
        x = x + prec_df['판결요지'][idx]
    if not pd.isna(prec_df['판례내용'][idx]):
        x = x + prec_df['판례내용'][idx]
    
    train_dataset.append(x)

  0%|          | 0/83520 [00:00<?, ?it/s]

### 2) Data 전처리

#### (1) 한글, 숫자 외의 문자 제거

In [5]:
x_df = pd.DataFrame(train_dataset, columns=['X'])
x_df.head()

,X
0,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당..."
1,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...
3,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극)【신청인(재심상대...
4,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...


In [6]:
x_df['X'] = x_df['X'].str.replace('[^가-힣0-9 ]', '')
x_df.head()

C:\Users\Park Ji Seong\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,X
0,피고인이 카카오톡 오픈채팅방을 통해 미성년자 여 11세을 알게 된 후 당시 13...
1,신청인 피항고인 신청인피신청인 항고인 피신청인제1심결정 서울가정법원 2019 6 1...
2,이 을 상대로 소송을 제기하자 이 과 소송위임계약을 체결하여 이 을 대리하여 소...
3,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부적극신청인재심상대방 재항...
4,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 특히 환자가 병...


#### (2) 형태소 토큰화
- Macab은 위키피디아 데이터로 학습된 형태소 분석기로 위키피디아로 사전학습된 fasttext를 학습하기 위한 전처리 과정에서 사용되는 형태소 분석기로 적합하다.

In [7]:
# Mecab 형태소 분석기 로드
mecab = Mecab('C:\mecab\mecab-ko-dic')

In [8]:
# mecab 형태소 분석기를 통한 형태소 토큰화
tokenized_vocab = []

for sent in tqdm(x_df['X']):
    token_sent = mecab.morphs(sent)
    tokenized_vocab.append(token_sent)

  0%|          | 0/83520 [00:00<?, ?it/s]

### 3) FastText Fine-Tunning

In [9]:
ko_models.build_vocab(tokenized_vocab, update=True) # Vocab 업데이트

In [10]:
ko_models.train(tokenized_vocab, total_examples=len(tokenized_vocab), epochs=10, compute_loss=True) # fine tunning

(324411063, 1617131550)

### 4) Evaluation FastText model
- https://github.com/SungjoonPark/KoreanWordVectors
    - 한국어 단어 유사도 셋으로 임베딩 성능 평가를 위한 아날로기 데이터

In [11]:
accuracy = ko_models.wv.evaluate_word_analogies('./embedding_evaluation/word_analogy_korean.txt')

In [20]:
accuracy[0], accuracy[1]

(0.13564668769716087,
 [{'section': 'sem1_capital-conturies',
   'correct': [('아테네', '그리스', '방콕', '태국'),
    ('아테네', '그리스', '코펜하겐', '덴마크'),
    ('아테네', '그리스', '스톡홀름', '스웨덴'),
    ('오슬로', '노르웨이', '스톡홀름', '스웨덴'),
    ('타이페이', '대만', '도쿄', '일본'),
    ('코펜하겐', '덴마크', '헬싱키', '핀란드'),
    ('코펜하겐', '덴마크', '키예프', '우크라이나'),
    ('코펜하겐', '덴마크', '리스본', '포르투갈'),
    ('코펜하겐', '덴마크', '런던', '영국'),
    ('코펜하겐', '덴마크', '마드리드', '스페인'),
    ('코펜하겐', '덴마크', '마닐라', '필리핀'),
    ('코펜하겐', '덴마크', '파리', '프랑스'),
    ('코펜하겐', '덴마크', '스톡홀름', '스웨덴'),
    ('코펜하겐', '덴마크', '도쿄', '일본'),
    ('베를린', '독일', '카이로', '이집트'),
    ('베를린', '독일', '코펜하겐', '덴마크'),
    ('베를린', '독일', '헬싱키', '핀란드'),
    ('베를린', '독일', '키예프', '우크라이나'),
    ('베를린', '독일', '리스본', '포르투갈'),
    ('베를린', '독일', '런던', '영국'),
    ('베를린', '독일', '마드리드', '스페인'),
    ('베를린', '독일', '마닐라', '필리핀'),
    ('베를린', '독일', '파리', '프랑스'),
    ('베를린', '독일', '스톡홀름', '스웨덴'),
    ('베를린', '독일', '도쿄', '일본'),
    ('모스크바', '러시아', '파리', '프랑스'),
    ('모스크바', '러시아', '스톡홀름', '스웨덴'),
    ('모스

In [22]:
evaluate_text = pd.read_csv("./embedding_evaluation/WS353_korean.csv", encoding='utf-8-sig')
evaluate_text.head()

,word 1,word 2,kor_score
0,사랑,섹스,6.42
1,호랑이,고양이,7.17
2,호랑이,호랑이,10.00
3,책,종이,6.17
4,컴퓨터,키보드,6.67


In [23]:
evaluate_text.to_csv("./embedding_evaluation/WS353_korean.txt", index=False, sep='\t')

In [24]:
accuracy_sim = ko_models.wv.evaluate_word_pairs('./embedding_evaluation/WS353_korean.txt')

In [29]:
accuracy_sim

((0.5674214271130962, 6.124248069512914e-30),
 SpearmanrResult(correlation=0.5655872921305093, pvalue=1.0226246937610297e-29),
 5.099150141643059)

### 5) save Fine-Tunning FastText model

In [30]:
ko_models.save("./fine_tuned_fasttext/fine_tune_fasttext")